In [1]:
#import all the necessary packages

import pandas as pd
import random

import nltk
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk import stem
stemmer = stem.PorterStemmer()
from nltk import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
import string
punct = list(string.punctuation)
from collections import Counter
import requests
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
import numpy as np
import praw
import datetime
import json
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aoife\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aoife\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import PyPDF2

In [3]:
# open the PDF file
policy = open(r'C:\Users\aoife\Documents\Capstone\Capstone Quant Policies\Surrey Disability.pdf', 'rb')

# create a PDF reader object
pdf_reader = PyPDF2.PdfReader(policy)

In [4]:
# read the text from each page
text = ''
for page_num in range(len(pdf_reader.pages)):
    page = pdf_reader.pages[page_num]
    text += page.extract_text()

# close the PDF file
policy.close()

In [5]:
# split the text into sentences
sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)

# create a dataframe with the sentences as rows
df = pd.DataFrame({'Sentence': sentences})

In [6]:
df

,Sentence
0,\n \n \n \nDisability and Neurodiver gence Pr...
1,\n Disclosable \nAssociated Policy \nState...
2,3 \n 3.2 What is Neurodiversity and Neurodive...
3,4 \n 3.3 Examples of disabilities and neurod...
4,4 \n 3.5 What is a Tailored Adjustment Plan?
...,...
172,"\nEquality, Diversity and Inclusion reports to..."
173,\n \n5.6 Legislative Context and Higher Educ...
174,\n \n5.8 Health and Safety \nHealth and safe...
175,\n \n6 Stakeholder Engagement and Equality ...


In [7]:
#check to see if any rows are empty
rows_with_nan = df[df.isna().any(axis=1)]

len(rows_with_nan)

0

In [8]:
#check the length of the dataframe
len(df)

177

In [9]:
# convert each row in the dataframe to a string and then convert the datadrame to a list of strings
p1 = df.apply(lambda row: ' '.join(row.astype(str)), axis=1).tolist()

In [10]:
p1

[' \n \n \n \nDisability and Neurodiver gence Procedure  \n \nEnabling Policy Statement ; \nExecutive Owner s; \nApproval Route:  \n Our Colleagues - Chief People Officer - Operations Committee  \n \nIs the Procedure for \ninternal use only (Non -\ndisclosable)?',
 ' \n   Disclosable  \nAssociated Policy \nStatements:    Our Students - Chief Student Officer   \nAuthorised  Owners: Director of HR  \n \nAuthorised Co -ordinator:  Head of HR Performance Delivery  \nEffective date: February  2024  \nDue date for full review:  January 2027 \nSub d ocumentation:  Tailored Adjustment Plan  and Guidance  \n \n \nApproval History \n \n \n \n \n \n \n  \n Version Reason for review  Approval Route  Date \n1.0 Policy out of date  and migration to POPP \ntemplate  Operations Committee  20 Feb 2024   \nDisability and Neurodiver gence  Procedure  \nVersion 1.0 \n2 \n Table of Contents  \n \nSection  \n Content  Page  \n1 Purpose  \n 3 \n2 Scope and Exceptions to the Procedure  \n 3 \n3 Definitions an

In [11]:
# define a function to remove punctuation and numbers from a string
def remove_punctuation_and_numbers(text):
    # define a regular expression pattern to match punctuation characters and numbers
    pattern = r'[^A-Za-z\s]'
    # replace punctuation characters and numbers with an empty string
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

# apply the previously created function to each string in the list
cleaned_p1 = [remove_punctuation_and_numbers(string) for string in p1]

# display the cleaned list of strings
print(cleaned_p1)

[' \n \n \n \nDisability and Neurodiver gence Procedure  \n \nEnabling Policy Statement  \nExecutive Owner s \nApproval Route  \n Our Colleagues  Chief People Officer  Operations Committee  \n \nIs the Procedure for \ninternal use only Non \ndisclosable', ' \n   Disclosable  \nAssociated Policy \nStatements    Our Students  Chief Student Officer   \nAuthorised  Owners Director of HR  \n \nAuthorised Co ordinator  Head of HR Performance Delivery  \nEffective date February    \nDue date for full review  January  \nSub d ocumentation  Tailored Adjustment Plan  and Guidance  \n \n \nApproval History \n \n \n \n \n \n \n  \n Version Reason for review  Approval Route  Date \n Policy out of date  and migration to POPP \ntemplate  Operations Committee   Feb    \nDisability and Neurodiver gence  Procedure  \nVersion  \n \n Table of Contents  \n \nSection  \n Content  Page  \n Purpose  \n  \n Scope and Exceptions to the Procedure  \n  \n Definitions and Terminology  \n  \n   What is Disability '

In [12]:
# tokenize each string in the list of strings
tokenized_p1 = [word_tokenize(string) for string in cleaned_p1]

# display the tokenized list of strings
print(tokenized_p1)

[['Disability', 'and', 'Neurodiver', 'gence', 'Procedure', 'Enabling', 'Policy', 'Statement', 'Executive', 'Owner', 's', 'Approval', 'Route', 'Our', 'Colleagues', 'Chief', 'People', 'Officer', 'Operations', 'Committee', 'Is', 'the', 'Procedure', 'for', 'internal', 'use', 'only', 'Non', 'disclosable'], ['Disclosable', 'Associated', 'Policy', 'Statements', 'Our', 'Students', 'Chief', 'Student', 'Officer', 'Authorised', 'Owners', 'Director', 'of', 'HR', 'Authorised', 'Co', 'ordinator', 'Head', 'of', 'HR', 'Performance', 'Delivery', 'Effective', 'date', 'February', 'Due', 'date', 'for', 'full', 'review', 'January', 'Sub', 'd', 'ocumentation', 'Tailored', 'Adjustment', 'Plan', 'and', 'Guidance', 'Approval', 'History', 'Version', 'Reason', 'for', 'review', 'Approval', 'Route', 'Date', 'Policy', 'out', 'of', 'date', 'and', 'migration', 'to', 'POPP', 'template', 'Operations', 'Committee', 'Feb', 'Disability', 'and', 'Neurodiver', 'gence', 'Procedure', 'Version', 'Table', 'of', 'Contents', 'Sec

In [13]:
# as some strings are now empty, remove empty strings from a list of strings
cleaned_list_of_strings = [string for string in tokenized_p1 if string]

# and just in case that doesn't work,, remove empty strings from a list of lists of strings
cleaned_tokenized_p1 = [[token for token in tokens if token] for tokens in tokenized_p1]

In [14]:
cleaned_tokenized_p1

[['Disability',
  'and',
  'Neurodiver',
  'gence',
  'Procedure',
  'Enabling',
  'Policy',
  'Statement',
  'Executive',
  'Owner',
  's',
  'Approval',
  'Route',
  'Our',
  'Colleagues',
  'Chief',
  'People',
  'Officer',
  'Operations',
  'Committee',
  'Is',
  'the',
  'Procedure',
  'for',
  'internal',
  'use',
  'only',
  'Non',
  'disclosable'],
 ['Disclosable',
  'Associated',
  'Policy',
  'Statements',
  'Our',
  'Students',
  'Chief',
  'Student',
  'Officer',
  'Authorised',
  'Owners',
  'Director',
  'of',
  'HR',
  'Authorised',
  'Co',
  'ordinator',
  'Head',
  'of',
  'HR',
  'Performance',
  'Delivery',
  'Effective',
  'date',
  'February',
  'Due',
  'date',
  'for',
  'full',
  'review',
  'January',
  'Sub',
  'd',
  'ocumentation',
  'Tailored',
  'Adjustment',
  'Plan',
  'and',
  'Guidance',
  'Approval',
  'History',
  'Version',
  'Reason',
  'for',
  'review',
  'Approval',
  'Route',
  'Date',
  'Policy',
  'out',
  'of',
  'date',
  'and',
  'migratio

In [15]:
#lemmatize
postlemma = []

for i in cleaned_tokenized_p1: 
    postlemma.append([lemmatizer.lemmatize(j) for j in i])

In [16]:
postlemma

[['Disability',
  'and',
  'Neurodiver',
  'gence',
  'Procedure',
  'Enabling',
  'Policy',
  'Statement',
  'Executive',
  'Owner',
  's',
  'Approval',
  'Route',
  'Our',
  'Colleagues',
  'Chief',
  'People',
  'Officer',
  'Operations',
  'Committee',
  'Is',
  'the',
  'Procedure',
  'for',
  'internal',
  'use',
  'only',
  'Non',
  'disclosable'],
 ['Disclosable',
  'Associated',
  'Policy',
  'Statements',
  'Our',
  'Students',
  'Chief',
  'Student',
  'Officer',
  'Authorised',
  'Owners',
  'Director',
  'of',
  'HR',
  'Authorised',
  'Co',
  'ordinator',
  'Head',
  'of',
  'HR',
  'Performance',
  'Delivery',
  'Effective',
  'date',
  'February',
  'Due',
  'date',
  'for',
  'full',
  'review',
  'January',
  'Sub',
  'd',
  'ocumentation',
  'Tailored',
  'Adjustment',
  'Plan',
  'and',
  'Guidance',
  'Approval',
  'History',
  'Version',
  'Reason',
  'for',
  'review',
  'Approval',
  'Route',
  'Date',
  'Policy',
  'out',
  'of',
  'date',
  'and',
  'migratio

In [17]:
#lower all cases
postfin1 = [[j.lower() for j in i] for i in postlemma]

In [18]:
postfin1

[['disability',
  'and',
  'neurodiver',
  'gence',
  'procedure',
  'enabling',
  'policy',
  'statement',
  'executive',
  'owner',
  's',
  'approval',
  'route',
  'our',
  'colleagues',
  'chief',
  'people',
  'officer',
  'operations',
  'committee',
  'is',
  'the',
  'procedure',
  'for',
  'internal',
  'use',
  'only',
  'non',
  'disclosable'],
 ['disclosable',
  'associated',
  'policy',
  'statements',
  'our',
  'students',
  'chief',
  'student',
  'officer',
  'authorised',
  'owners',
  'director',
  'of',
  'hr',
  'authorised',
  'co',
  'ordinator',
  'head',
  'of',
  'hr',
  'performance',
  'delivery',
  'effective',
  'date',
  'february',
  'due',
  'date',
  'for',
  'full',
  'review',
  'january',
  'sub',
  'd',
  'ocumentation',
  'tailored',
  'adjustment',
  'plan',
  'and',
  'guidance',
  'approval',
  'history',
  'version',
  'reason',
  'for',
  'review',
  'approval',
  'route',
  'date',
  'policy',
  'out',
  'of',
  'date',
  'and',
  'migratio

In [19]:
# replace the old comments with the new pre-processed comments
data = {'Sentence': postfin1}
p1 = pd.DataFrame(data)

In [20]:
p1

,Sentence
0,"[disability, and, neurodiver, gence, procedure..."
1,"[disclosable, associated, policy, statements, ..."
2,"[what, is, neurodiversity, and, neurodivergent]"
3,"[examples, of, disability, and, neurodivergenc..."
4,"[what, is, a, tailored, adjustment, plan]"
...,...
172,"[equality, diversity, and, inclusion, report, ..."
173,"[legislative, context, and, higher, education,..."
174,"[health, and, safety, health, and, safety, imp..."
175,"[stakeholder, engagement, and, equality, impac..."


In [21]:
# set display options to show all columns and rows in the dataframe so i can quickly scan through
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

# display the dataframe
print(p1)

                                              Sentence
0    [disability, and, neurodiver, gence, procedure...
1    [disclosable, associated, policy, statements, ...
2      [what, is, neurodiversity, and, neurodivergent]
3    [examples, of, disability, and, neurodivergenc...
4            [what, is, a, tailored, adjustment, plan]
5                               [what, is, accessable]
6    [procedural, principles, who, is, responsible,...
7    [disability, and, neurodivergence, equality, f...
8    [a, key, principle, of, this, vision, is, to, ...
9    [our, value, of, inclusion, inspiration, innov...
10   [we, continue, to, strive, for, surrey, to, be...
11   [the, objective, of, the, procedure, is, to, e...
12   [we, believe, in, the, need, for, equity, whic...
13   [whilst, equality, mean, providing, the, same,...
14   [this, procedure, recognises, that, disabled, ...
15   [it, take, a, it, starting, point, the, premis...
16   [the, broad, procedure, aim, are, based, on, t...
17   [to, 

In [22]:
# Doing so helped me notice that there are a bunch of empty lists, so we will remove those now
# find rows with empty lists in the 'Sentence' column
rows_with_empty_lists = p1[p1['Sentence'].apply(len) == 0]

# display rows with empty lists
print(rows_with_empty_lists)

Empty DataFrame
Columns: [Sentence]
Index: []


In [23]:
# remove rows with empty lists in the 'Sentence' column
p1 = p1[p1['Sentence'].apply(len) > 0]

# reset index
p1.reset_index(drop=True, inplace=True)

# display the modified DataFrame
print(p1)

                                              Sentence
0    [disability, and, neurodiver, gence, procedure...
1    [disclosable, associated, policy, statements, ...
2      [what, is, neurodiversity, and, neurodivergent]
3    [examples, of, disability, and, neurodivergenc...
4            [what, is, a, tailored, adjustment, plan]
5                               [what, is, accessable]
6    [procedural, principles, who, is, responsible,...
7    [disability, and, neurodivergence, equality, f...
8    [a, key, principle, of, this, vision, is, to, ...
9    [our, value, of, inclusion, inspiration, innov...
10   [we, continue, to, strive, for, surrey, to, be...
11   [the, objective, of, the, procedure, is, to, e...
12   [we, believe, in, the, need, for, equity, whic...
13   [whilst, equality, mean, providing, the, same,...
14   [this, procedure, recognises, that, disabled, ...
15   [it, take, a, it, starting, point, the, premis...
16   [the, broad, procedure, aim, are, based, on, t...
17   [to, 

In [24]:
# just to check it is still all good in this format too (can't be too careful)
p1

,Sentence
0,"[disability, and, neurodiver, gence, procedure..."
1,"[disclosable, associated, policy, statements, ..."
2,"[what, is, neurodiversity, and, neurodivergent]"
3,"[examples, of, disability, and, neurodivergenc..."
4,"[what, is, a, tailored, adjustment, plan]"
5,"[what, is, accessable]"
6,"[procedural, principles, who, is, responsible,..."
7,"[disability, and, neurodivergence, equality, f..."
8,"[a, key, principle, of, this, vision, is, to, ..."
9,"[our, value, of, inclusion, inspiration, innov..."


In [25]:
# save the dataset to be able to use in analysis

#export to excel file
p1.to_excel('p7.xlsx')

#export to csv file
p1.to_csv('p7.csv', index=False)